## Testes da API

Nesse notebook são feitos alguns testes com a API do Olho Vivo

In [1]:
import pandas as pd
from requests import Session
from typing import Optional

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

#espera = 30
for i in range(1):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    #time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '13:40',
  'l': [{'c': '978A-10',
    'cl': 34137,
    'sl': 2,
    'lt0': 'METRÔ BARRA FUNDA',
    'lt1': 'TERM. CACHOEIRINHA',
    'qv': 6,
    'vs': [{'p': 12475,
      'a': True,
      'ta': '2025-09-16T16:40:07Z',
      'py': -23.481173,
      'px': -46.674104,
      'sv': None,
      'is': None},
     {'p': 12594,
      'a': True,
      'ta': '2025-09-16T16:40:26Z',
      'py': -23.475101000000002,
      'px': -46.670268,
      'sv': None,
      'is': None},
     {'p': 12467,
      'a': True,
      'ta': '2025-09-16T16:40:28Z',
      'py': -23.499009,
      'px': -46.68539625,
      'sv': None,
      'is': None},
     {'p': 12471,
      'a': True,
      'ta': '2025-09-16T16:40:15Z',
      'py': -23.513153,
      'px': -46.6905325,
      'sv': None,
      'is': None},
     {'p': 12433,
      'a': True,
      'ta': '2025-09-16T16:40:12Z',
      'py': -23.5252075,
      'px': -46.66526175,
      'sv': None,
      'is': None},
     {'p': 12434,
      'a': True,
      'ta': '2

In [6]:
len(posicoes)

1

In [7]:
rows = []

for h in posicoes:
    if h:
        for l in h['l']:

            for veiculo in l['vs']:

                row = {
                    'linha': l['c'],
                    'id_onibus': veiculo['p'],
                    'timestamp': veiculo['ta'],
                    'lat': veiculo['py'],
                    'lon': veiculo['px'] 
                }

                rows.append(row)

df_posicoes = pd.DataFrame(rows)

Conversão de horário

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-16T16:40:07Z,-23.481173,-46.674104
1,978A-10,12594,2025-09-16T16:40:26Z,-23.475101,-46.670268
2,978A-10,12467,2025-09-16T16:40:28Z,-23.499009,-46.685396
3,978A-10,12471,2025-09-16T16:40:15Z,-23.513153,-46.690533
4,978A-10,12433,2025-09-16T16:40:12Z,-23.525208,-46.665262
...,...,...,...,...,...
9820,6020-10,66902,2025-09-16T16:40:20Z,-23.826263,-46.744140
9821,1775-10,26466,2025-09-16T16:39:58Z,-23.458248,-46.615258
9822,4008-21,48384,2025-09-16T16:40:04Z,-23.579354,-46.432372
9823,6817-10,72001,2025-09-16T16:40:05Z,-23.681794,-46.781263


In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

df_posicoes['timestamp'] = df_posicoes['timestamp'].dt.tz_convert("America/Sao_Paulo")

df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-16 13:40:07-03:00,-23.481173,-46.674104
1,978A-10,12594,2025-09-16 13:40:26-03:00,-23.475101,-46.670268
2,978A-10,12467,2025-09-16 13:40:28-03:00,-23.499009,-46.685396
3,978A-10,12471,2025-09-16 13:40:15-03:00,-23.513153,-46.690533
4,978A-10,12433,2025-09-16 13:40:12-03:00,-23.525208,-46.665262
...,...,...,...,...,...
9820,6020-10,66902,2025-09-16 13:40:20-03:00,-23.826263,-46.744140
9821,1775-10,26466,2025-09-16 13:39:58-03:00,-23.458248,-46.615258
9822,4008-21,48384,2025-09-16 13:40:04-03:00,-23.579354,-46.432372
9823,6817-10,72001,2025-09-16 13:40:05-03:00,-23.681794,-46.781263


In [10]:
horas = df_posicoes["timestamp"].dt.hour

horas_unicas = horas.unique()
horas_unicas.sort()

print(horas_unicas)

[13]


Ônibus duplicados

In [11]:
duplicados = (df_posicoes.groupby(["id_onibus", "timestamp"]).filter(lambda x: len(x) > 1).sort_values(["id_onibus", "timestamp"]))

duplicados

,linha,id_onibus,timestamp,lat,lon
